In [ ]:
pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [ ]:
from transformers import (
    GPT2Tokenizer,
    GPT2ForSequenceClassification,  # Use sequence classification model
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)
import numpy as np
import evaluate
import os
from datasets import load_dataset
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Load accuracy metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Step 1: Load the Yelp Review Dataset
dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")

# Step 2: Initialize the GPT-2 Tokenizer and Assign a Padding Token
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Use <|endoftext|> as the padding token

# Step 3: Tokenize the Dataset and Add Labels
def tokenize_function(example):
    tokens = tokenizer(
        example['text'],             # Process the text column
        truncation=True,             # Truncate to max length
        padding='max_length',        # Pad to max length
        max_length=1024,             # Define maximum input length
    )
    tokens["labels"] = example['label']  # Use label from dataset for classification task
    return tokens

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Select a subset of the dataset
small_train_dataset = tokenized_dataset["train"].shuffle(seed=42).select(range(2000))
small_eval_dataset = tokenized_dataset["test"].shuffle(seed=42).select(range(1000))

# Step 4: Define the Model for Sequence Classification
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=2)
model.config.pad_token_id = tokenizer.pad_token_id  # Assign the padding token to the model

# Step 5: Create a Data Collator for Padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# Step 6: Define the Training Arguments
training_args = TrainingArguments(
    output_dir='test_train_gpt2',           # Directory to save checkpoints
    eval_strategy="epoch",                  # Perform evaluation at the end of each epoch
    per_device_train_batch_size=8,          # Batch size per device
    per_device_eval_batch_size=8,           # Evaluation batch size
    num_train_epochs=10,                    # Number of training epochs
    learning_rate=0.00005,
    prediction_loss_only=False,
)

# Step 7: Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,      # Training dataset
    eval_dataset=small_eval_dataset,        # Evaluation dataset
    tokenizer=tokenizer,                    # Tokenizer
    data_collator=data_collator,            # Data collator for padding
    compute_metrics=compute_metrics,        # Compute accuracy metrics
)

# Step 8: Train the Model
trainer.train()

# Step 9: Save the Fine-Tuned Model
model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-4-e4e547ca8e83>:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.368520,0.850000
2,0.482700,0.368130,0.831000
3,0.482700,0.470828,0.847000
4,0.234100,0.814017,0.840000
5,0.234100,1.227059,0.821000
6,0.085200,1.160990,0.840000
7,0.085200,1.174301,0.839000
8,0.022700,1.460478,0.844000
9,0.022700,1.369844,0.834000
10,0.011700,1.382166,0.837000


('./fine_tuned_gpt2/tokenizer_config.json',
 './fine_tuned_gpt2/special_tokens_map.json',
 './fine_tuned_gpt2/vocab.json',
 './fine_tuned_gpt2/merges.txt',
 './fine_tuned_gpt2/added_tokens.json')

In [ ]:
from transformers import GPT2ForSequenceClassification

# 加载训练好的分类模型
model = GPT2ForSequenceClassification.from_pretrained('./fine_tuned_gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('./fine_tuned_gpt2')

# 输入文本
input_text = "This is a great product!"

# 编码文本
inputs = tokenizer(input_text, return_tensors='pt')

# 获取模型的输出
outputs = model(**inputs)

# 获取预测结果
logits = outputs.logits
predicted_class = logits.argmax(dim=-1).item()

print(f"Predicted class: {predicted_class}")


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at ./fine_tuned_gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted class: 1


In [ ]:
from datasets import load_dataset
dataset = load_dataset("cornell-movie-review-data/rotten_tomatoes")
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length",max_length=512, truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)#将 tokenize_function 应用到 dataset 的每个元素上。dataset 应该是一个 Hugging Face datasets 库中的数据集对象。

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(2000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=2)
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch",num_train_epochs=10,per_device_train_batch_size=8,learning_rate=0.00005)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)
trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.46k [00:00<?, ?B/s]

train.parquet:   0%|          | 0.00/699k [00:00<?, ?B/s]

validation.parquet:   0%|          | 0.00/90.0k [00:00<?, ?B/s]

test.parquet:   0%|          | 0.00/92.2k [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.430857,0.799000
2,0.386700,0.800473,0.814000
3,0.386700,0.873296,0.824000
4,0.068400,1.222997,0.826000
5,0.068400,1.205391,0.831000
6,0.011600,1.344945,0.829000
7,0.011600,1.386298,0.839000
8,0.003300,1.440656,0.833000
9,0.003300,1.462807,0.832000
10,0.000000,1.469872,0.832000


TrainOutput(global_step=2500, training_loss=0.09400564101934433, metrics={'train_runtime': 539.3233, 'train_samples_per_second': 37.084, 'train_steps_per_second': 4.635, 'total_flos': 5262221107200000.0, 'train_loss': 0.09400564101934433, 'epoch': 10.0})

In [ ]:
pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 117.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 10.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import gradio as gr
from transformers import BertTokenizer, BertForSequenceClassification, GPT2Tokenizer, GPT2ForSequenceClassification
import torch

# Step 1: Load BERT and GPT-2 models for sentiment classification

# Load BERT model and tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Load GPT-2 model and tokenizer
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)

# Step 2: Define inference functions for each model

# BERT sentiment classification
def classify_with_bert(text):
    inputs = bert_tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    return "Positive" if prediction == 1 else "Negative"

# GPT-2 sentiment classification
def classify_with_gpt2(text):
    inputs = gpt2_tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=1024)
    with torch.no_grad():
        outputs = gpt2_model(**inputs)
    logits = outputs.logits
    prediction = torch.argmax(logits, dim=-1).item()
    return "Positive" if prediction == 1 else "Negative"

# Step 3: Define Gradio interface

def analyze_sentiment(text):
    bert_result = classify_with_bert(text)
    gpt2_result = classify_with_gpt2(text)
    return bert_result, gpt2_result

# Create a Gradio interface
iface = gr.Interface(
    fn=analyze_sentiment,
    inputs=gr.Textbox(label="Enter Text for Sentiment Analysis"),
    outputs=[gr.Textbox(label="BERT Sentiment"), gr.Textbox(label="GPT-2 Sentiment")],
    live=False,
    title="Sentiment Analysis with BERT and GPT-2",
    description="This interface analyzes the sentiment of the input text using both BERT and GPT-2 models."

)

# Step 4: Launch the interface
iface.launch()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://996f0f2fac6d1900a8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
